In [ ]:
# Imports
using Gurobi
using LinearAlgebra
using JuMP
using Random
using LaTeXStrings
include("fit.jl");
include("convexRegress.jl");
include("bin_to_leaves.jl");

In [ ]:
# Generating monomial data
a = 1; b = 2; 
x = range(1, stop=20, step=1);
y = range(1, stop=20, step=1);
X = zeros(size(x,1)*size(y,1), 2)
for i=1:size(x,1)
    for j=1:size(y,1)
        X[(i-1)*size(y,1)+j, :] = [x[i], y[j]];
    end
end
Y = 76 .*X[:,1].^0.32./X[:,2].^0.7;

In [ ]:
# Plotting data along two X-axes, in original space
using Plots
plt3d = Plots.plot(X[:,2], X[:,1], Y[:,1],
     seriestype=:surface, markersize = 2, camera=(20,30))
xlabel!("x_1"); ylabel!("x_2"); title!(L"$y = 76x_1^{0.32}x_2^{-0.7}$")
display(plt3d)

In [ ]:
# Plotting data in log-space
using Plots
u = log.(X);
v = log.(Y);
plt3d = Plots.plot(u[:,1], u[:,2], v[:,1],
     seriestype=:surface, markersize = 2)
xlabel!("u_1=log(x_1)"); ylabel!("u_2=log(x_2)"); title!(L"$v = log(y) = log(76e^{0.32u_1-0.7u_2})$")
display(plt3d)

In [ ]:
bs, b0, ts, ss = regress(v, u, 0.00001, 0, 100.)

In [ ]:
# Plotting actual vs. predictions
plt3d = Plots.plot(u[:,1], u[:,2], v[:,1],
     seriestype=:surface, markersize = 2);
Plots.plot!(u[:,1], u[:,2], u*bs .+ b0,
     seriestype=:scatter, markersize = 2);
display(plt3d)

In [ ]:
# Solving for...
thetas, ksis = convexRegress(v,u,100);

In [ ]:
# Plotting actual vs. predictions
plt3d = Plots.plot(u[:,1], u[:,2], v[:,1],
     seriestype=:scatter, markersize = 2);
Plots.plot!(u[:,1], u[:,2], thetas,
     seriestype=:scatter, markersize = 2);
display(plt3d)


In [ ]:
# Now let's try coupling these methods with Optimal Regression Trees!
# First with parallel splits
lnr = IAI.OptimalTreeRegressor(random_seed=1, max_depth=3, cp=0.001, minbucket=0.05, regression_sparsity=:all)
IAI.fit!(lnr, u, v)


In [ ]:
# Plotting actual vs. predictions
vpredict = IAI.predict(lnr, u)
leaf_index, leaves = bin_to_leaves(lnr, u)
display(Plots.plot(u[:,1], u[:,2], vpredict,
     seriestype=:scatter, markersize = 2, zcolor=leaf_index, seriescolor=:phase, camera = (15, 30));)